## 探索
加载数据

In [ ]:
import para
import csv
import numpy as np
import matplotlib.pyplot as plt

# print(t)
for i in range(para.machine_num):
    file_path = para.train_data +  '{0:03d}'.format(i+2) + '/201807.csv'
    with open(file_path,'r') as file:
        data = csv.reader(file)
        data.__next__()
        full_data = []
        p_miss_data = []
        for record in data:
            if any([r==''for r in record]):
                p_miss_data.append(record)
            else:
                full_data.append(record)
        raw_data_str = full_data
    
    break
    
# record to be filled
file_path = para.train_data+'template_submit_result.csv'
all_miss_data = []
with open(file_path,'r') as file:
    data = csv.reader(file)
    data.__next__()
    for record in data:
        if record[1] == '1':
            all_miss_data.append(record)

无缺失字段记录

In [ ]:
%matplotlib inline
t = ['M'] + ['i4'] + ['f4']*15+['i4']+['f4']*3+['i4']+['f4']*26+['i4']+['f4']*5+['i4']+['f4']*12+['i4']+['f4']*2

raw_data_str = np.array(raw_data_str)
raw_data = []

for i in range(70):
    s = raw_data_str[:,i].astype(t[i])
    if i != 0 and i !=1:
        plt.figure()
        plt.hist(s)
        plt.xlabel('var num {0}, type {5}, min {1:.2f}, max {2:.2f}, mean {3:.2f}, std {4:.2f}'.format(i-1,np.min(s),np.max(s),np.mean(s),np.std(s),t[i]))
        plt.show()
    raw_data.append(s)
#     print(s[0],s.dtype)

# print(raw_data[0],raw_data.dtype)

时序性测试

In [ ]:
# plt.figure(figsize=(16,8))
for i in range(2,70):
    plt.plot(raw_data[i],label=str(i-1))
    plt.xlabel('var'+str(i-1))
    plt.show()

有缺失字段记录

In [ ]:
%matplotlib inline
p_miss_count = [0 for _ in range(70)]
p_miss_total = len(p_miss_data)
# print(p_miss_total)
for r in p_miss_data:
    for i in range(len(r)):
        if r[i]=='':
            p_miss_count[i]+=1
p_miss_count = np.array(p_miss_count)
plt.figure(figsize=(16,8))
plt.bar(np.array(range(70)),p_miss_count/p_miss_total)
plt.show()

f_miss_t = []    
p_miss_t = []
for record in all_miss_data:
    flag = False
    for p_r in p_miss_data:
        if record[0]==p_r[0]:
            flag = True
            break
    if flag:
        p_miss_t.append(record[0])
    else:
        f_miss_t.append(record[0])    
plt.figure(figsize=(16,8))
# plt.plot(raw_data[0],[1 for _ in range(len(raw_data[0]))],'b.',label='full')
plt.plot(np.array(p_miss_t,dtype='M'),[0.9 for _ in range(len(p_miss_t))],'r.',label='p_miss')
plt.plot(np.array(f_miss_t,dtype='M'),[0.8 for _ in range(len(f_miss_t))],'g.',label='f_miss')
plt.legend(loc='best')
plt.show()
    

预处理
1. 对float类型做standardization (x-mean)/std 
2. 对int，book类型做one_hot

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
# class used for pre-process data 
class preproc:
    def __init__(self, data, types):
        self.features = [True if t=='f4' else False for t in types] # 68个字段预处理分类操作
        self.continuous, self.discrete = self.classify(data)
        self.proc_cont = self.standardization(self.continuous)
        self.proc_disc = self.one_hot(self.discrete)
        
    def classify(self, data):
        return data[:,self.features], data[:,np.logical_not(self.features)]
        
    def standardization(self, data):
        self.stda = StandardScaler()
        return self.stda.fit_transform(data)
        
    def one_hot(self,data):
        self.enc = OneHotEncoder()
        return self.enc.fit_transform(data)
        

In [ ]:
#test
data = np.array(raw_data).T[:,2:]
pp = preproc(data,t[2:])
inputs = np.concatenate((pp.proc_cont,pp.proc_disc.todense()),axis=1)
print(inputs.shape)


## 实验

In [ ]:
%run exp.py

In [ ]:
from utilities import *
import pickle
import torch
dl = DataLoader()
machine_num = 0 # 0号为001，1号为002，以此类推
import para
raw_data = dl(para.train_data,machine_num) #加载数据

data = raw_data[:,1:] #移除时间列
with open("meta.pkl",'rb') as file:
    pp = pickle.loads(file.read())
    print('加载预处理meta完成。',flush=True)


In [ ]:
inputs = pp.transform(torch.tensor(data[0:10].astype('f4'))) #预处理输出
print(inputs.shape)

In [ ]:
import torch
import torch.nn as nn
rnn = nn.LSTM(input_size=141, hidden_size=256, bidirectional=True) #input_size, hidden_size, num_layers
# input: seq_len, batch, input_size
inputs = inputs.reshape(10,1,141).float()
h0 = torch.randn(2, 1, 256)#num_layers * num_directions, batch, hidden_size
c0 = torch.randn(2, 1, 256)#num_layers * num_directions, batch, hidden_size
output, (hn, cn) = rnn(inputs, (h0, c0)) # output: seq_len, batch, num_directions * hidden_size

In [ ]:
print(output.shape)

In [ ]:
rnn = nn.LSTMCell(141, 256)
hx = torch.randn(1, 256)
cx = torch.randn(1, 256)
output = []
for i in range(10):
    hx, cx = rnn(inputs[i], (hx, cx))
    output.append(hx)
    print(hx.shape)
print(output)

In [ ]:
import para
i = 1
file_path = para.train_data +  '{0:03d}'.format(i) + '/201807.csv'
with open(file_path,'r') as file:
    with open('small.csv','w') as output:
        i = 0
        while i<201:
            output.write(file.readline()+'\n')
            i+=1
print('Done')

In [ ]:
import numpy as np
data = np.loadtxt('small.csv',dtype=np.str,delimiter=',',skiprows=1)
print(data[0])
data = torch.tensor(data[:,1:].astype('f4'))

In [ ]:
from utilities import *
import pickle
import torch
print(data.shape)
with open("meta.pkl",'rb') as file:
    pp = pickle.loads(file.read())
print('加载预处理meta完成。',flush=True)
inputs = pp.transform(data)
print(inputs[0])


In [ ]:
inputs = inputs.view(-1,2,inputs.shape[-1]).float() # L x B x I

In [ ]:
import torch
import torch.nn as nn
rnn = nn.LSTMCell(input_size=141, hidden_size=256, bidirectional=True) #input_size, hidden_size, num_layers
# input: seq_len, batch, input_size
# inputs = inputs.reshape(10,1,141).float()
h0 = torch.randn(2, 2, 256)#num_layers * num_directions, batch, hidden_size
c0 = torch.randn(2, 2, 256)#num_layers * num_directions, batch, hidden_size
o = []
for i in range(l):
    if mask[i]:
        o.append(rnn(inputs[i,:,:]))
    else:
        o.append(rnn(o[-1]))

    
output, (hn, cn) = rnn(inputs, (h0, c0)) # output: seq_len, batch, num_directions * hidden_size

In [ ]:
print(output.shape)
net = nn.Sequential(
    nn.Linear(512, 141),
)
pred = net(output)
loss_func = torch.nn.MSELoss()
loss = loss_func(pred,inputs)
optimizer = torch.optim.Adam(, lr=0.001)
net.backward()

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.rnn_fw = torch.nn.LSTMCell(input_size=n_feature, hidden_size=n_hidden)
        self.rnn_bw = torch.nn.LSTMCell(input_size=n_feature, hidden_size=n_hidden)
        
        self.predict = torch.nn.Linear(n_hidden*2, n_output)
        
    def merge(self,missing,pred,mask):
        return missing + pred * mask
        
    def forward(self, x, is_train = True, mask = None):
        #x : L x B x F
        x_bw = x[::-1]
        out = []
        hx_fw = torch.randn(x.shape[1], x.shape[2]) # B x H
        cx_fw = torch.randn(x.shape[1], x.shape[2])
        hx_bw = torch.randn(x.shape[1], x.shape[2])
        cx_bw = torch.randn(x.shape[1], x.shape[2])
        if is_train:
            for i in range(x.shape[0]):
                hx_fw,cs_fw = self.rnn_fw(x[i],hx_fw,cs_fw)
                hx_bw,cs_bw = self.rnn_bw(x_bw[i],hx_bw,cs_bw)
                out.append(self.predict(torch.cat([hx_fw,hx_bw])))
        else:
            for i in range(x.shape[0]):
                if not mask[i]: # 正常值
                    hx_fw,cs_fw = self.rnn_fw(x[i],hx_fw,cs_fw)
                else: #缺失值
                    merge = self.merge(x[i],out[-1])
                    hx_fw,cs_fw = self.rnn_fw(x[i],hx_fw,cs_fw)
                if not mask[::-1][i]:
                    hx_bw,cs_bw = self.rnn_bw(x[i],hx_bw,cs_bw)
                else: #缺失值
                    merge = self.merge(x_bw[i],out[-1])
                    hx_fw,cs_fw = self.rnn_fw(x[i],hx_fw,cs_fw)
                hx_bw,cs_bw = self.rnn_bw(x_bw[i],hx_bw,cs_bw)
                out.append(self.predict(torch.cat([hx_fw,hx_bw])))
        out = torch.tensor(out)
        return out

### Seq2Seq test

In [ ]:
%run RNN/exp2.py

In [ ]:
%run RNN/evalute2.py

In [ ]:
%run check_submit.py

In [ ]:
%%time
%run utilities.py